In [119]:
import requests
import pandas as pd
import json

In [2]:
# Esta funcion imprime las 10 monedas volatiles con mas capitalizacion y luego las 10 monedas estables.
def get_top_coins():
    stable = []
    volatil = []

    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 100,
        "page": 1
    }

    response = requests.get(url, params=params)
    data = response.json()

    for coin in data:
        if coin['current_price'] > 1 or 0.99 > coin['current_price'] and len(volatil) < 10:
            volatil.append(coin['id'])
        elif 1.1 > coin['current_price'] > 0.97 and len(stable) < 10:
            stable.append(coin['id'])
        else:
            break
        
    return stable,volatil

stable, volatil = get_top_coins()
print(stable)
print(volatil)

['tether', 'usd-coin']
['bitcoin', 'ethereum', 'binancecoin', 'ripple', 'staked-ether', 'cardano', 'dogecoin', 'solana', 'tron', 'polkadot']


In [27]:
# Utiliza una lista de nombres de criptomonedas y a traves de la api de coingecko extrae la informacion de cada una de ellas para luego de un ETL guardar 
# la informacion en un archivo csv.
def get_historical_data(coinnamelist):
    for coin in coinnamelist:
        url = f"https://api.coingecko.com/api/v3/coins/{coin}/market_chart"
        params = {
            'vs_currency': 'usd',
            'days': '2160',
            'interval': 'daily'
        }
    
        response = requests.get(url, params=params)
    
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)
            df['market_caps'] = df['market_caps'].apply(lambda x: int(x[1]))
            df['total_volumes'] = df['total_volumes'].apply(lambda x: int(x[1]))
            df['Date'] = df['prices'].apply(lambda x: pd.to_datetime(x[0], unit='ms'))
            df['prices'] = df['prices'].apply(lambda x: int(x[1]))
            df.rename(columns={'prices': 'Price_usd','total_volumes':'Volume','market_caps':'Market_cap'}, inplace=True)
            df = df.groupby(df["Date"].dt.to_period("M")).tail(1)
            df.to_csv(f'./Data/{coin}.csv',index=False)
        else:
            print(f"Error al obtener los datos de {coin}:", response.status_code)


In [28]:
get_historical_data(volatil)

In [3]:
df.head()

NameError: name 'df' is not defined

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2161 entries, 0 to 2160
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Prices_usd   2161 non-null   int64         
 1   Market_caps  2161 non-null   int64         
 2   Volume       2161 non-null   int64         
 3   Date         2161 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 67.7 KB


In [103]:
url = 'https://api.coingecko.com/api/v3/global'
response = requests.get(url)
if response.status_code == 200:
    data = response.json()

In [104]:
data

{'data': {'active_cryptocurrencies': 10141,
  'upcoming_icos': 0,
  'ongoing_icos': 49,
  'ended_icos': 3376,
  'markets': 832,
  'total_market_cap': {'btc': 41826617.512654126,
   'eth': 666146378.5758387,
   'ltc': 16767600760.767443,
   'bch': 5819976931.995338,
   'bnb': 5102289699.125557,
   'eos': 1877271409650.8564,
   'xrp': 2083482612313.4902,
   'xlm': 8981386714414.314,
   'link': 177061633538.24048,
   'dot': 246721251014.45306,
   'yfi': 195197410.04602554,
   'usd': 1089245904598.5288,
   'aed': 4000838331197.057,
   'ars': 381194784189701.0,
   'aud': 1690771062954.0193,
   'bdt': 118999083561517.8,
   'bhd': 410680561902.5926,
   'bmd': 1089245904598.5288,
   'brl': 5379023050678.895,
   'cad': 1474965307351.3408,
   'chf': 958814153752.3783,
   'clp': 947153776343650.6,
   'cny': 7838758152443.305,
   'czk': 24159800937766.754,
   'dkk': 7479276505040.448,
   'eur': 1003500466988.5336,
   'gbp': 854950199765.2914,
   'hkd': 8537316603718.168,
   'huf': 384070629744202.

In [105]:
data = data['data']
df_global = pd.DataFrame(data)
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10141,0,49,3376,832,4.182662e+07,1.635032e+06,46.538128,-0.31574,1692759917
eth,10141,0,49,3376,832,6.661464e+08,2.604013e+07,18.039677,-0.31574,1692759917
ltc,10141,0,49,3376,832,1.676760e+10,6.554573e+08,NaN,-0.31574,1692759917
bch,10141,0,49,3376,832,5.819977e+09,2.275070e+08,NaN,-0.31574,1692759917
bnb,10141,0,49,3376,832,5.102290e+09,1.994521e+08,3.014816,-0.31574,1692759917


In [106]:
df_global.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67 entries, btc to sol
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   active_cryptocurrencies               67 non-null     int64  
 1   upcoming_icos                         67 non-null     int64  
 2   ongoing_icos                          67 non-null     int64  
 3   ended_icos                            67 non-null     int64  
 4   markets                               67 non-null     int64  
 5   total_market_cap                      61 non-null     float64
 6   total_volume                          61 non-null     float64
 7   market_cap_percentage                 10 non-null     float64
 8   market_cap_change_percentage_24h_usd  67 non-null     float64
 9   updated_at                            67 non-null     int64  
dtypes: float64(4), int64(6)
memory usage: 5.8+ KB


In [107]:
df_global['market_cap_percentage'].sum()

83.77842307714631

In [32]:
df_global['updated_at'] = df_global['updated_at'].apply(lambda x: pd.to_datetime(x, unit='s'))

In [33]:
df_global.head()

,active_cryptocurrencies,upcoming_icos,ongoing_icos,ended_icos,markets,total_market_cap,total_volume,market_cap_percentage,market_cap_change_percentage_24h_usd,updated_at
btc,10135,0,49,3376,832,4.180181e+07,1.598513e+06,46.565582,-0.586926,2023-08-23 01:04:55
eth,10135,0,49,3376,832,6.665769e+08,2.549009e+07,18.032160,-0.586926,2023-08-23 01:04:55
ltc,10135,0,49,3376,832,1.680649e+10,6.426848e+08,NaN,-0.586926,2023-08-23 01:04:55
bch,10135,0,49,3376,832,5.825016e+09,2.227503e+08,NaN,-0.586926,2023-08-23 01:04:55
bnb,10135,0,49,3376,832,5.086336e+09,1.945029e+08,3.020624,-0.586926,2023-08-23 01:04:55


In [42]:
delete_columns = []

In [35]:
df_global['active_cryptocurrencies'].unique()

array([10135], dtype=int64)

In [43]:
delete_columns.append('active_cryptocurrencies')

In [36]:
df_global['upcoming_icos'].unique()

array([0], dtype=int64)

In [44]:
delete_columns.append('upcoming_icos')

In [37]:
df_global['ongoing_icos'].unique()

array([49], dtype=int64)

In [45]:
delete_columns.append('ongoing_icos')

In [38]:
df_global['ended_icos'].unique()

array([3376], dtype=int64)

In [46]:
delete_columns.append('ended_icos')

In [39]:
df_global['markets'].unique()

array([832], dtype=int64)

In [47]:
delete_columns.append('markets')

In [40]:
df_global['market_cap_change_percentage_24h_usd'].unique()

array([-0.58692601])

In [48]:
delete_columns.append('market_cap_change_percentage_24h_usd')

In [41]:
df_global['updated_at'].unique()

array(['2023-08-23T01:04:55.000000000'], dtype='datetime64[ns]')

In [49]:
delete_columns.append('updated_at')

In [50]:
df_global.drop(delete_columns,axis=1,inplace=True)

In [51]:
df_global.head()

,total_market_cap,total_volume,market_cap_percentage
btc,4.180181e+07,1.598513e+06,46.565582
eth,6.665769e+08,2.549009e+07,18.032160
ltc,1.680649e+10,6.426848e+08,NaN
bch,5.825016e+09,2.227503e+08,NaN
bnb,5.086336e+09,1.945029e+08,3.020624


In [55]:
df_global = df_global.sort_values(by='market_cap_percentage', ascending=False)

In [58]:
df_global.fillna(0)

,total_market_cap,total_volume,market_cap_percentage
btc,4.180181e+07,1.598513e+06,46.565582
eth,6.665769e+08,2.549009e+07,18.032160
usdt,0.000000e+00,0.000000e+00,7.603567
bnb,5.086336e+09,1.945029e+08,3.020624
xrp,2.095904e+12,8.014796e+10,2.520860
...,...,...,...
xdr,8.181760e+11,3.128728e+10,0.000000
xag,4.642986e+10,1.775491e+09,0.000000
xau,5.730264e+08,2.191269e+07,0.000000
bits,4.180181e+13,1.598513e+12,0.000000


In [60]:
df_global.head(10)

,total_market_cap,total_volume,market_cap_percentage
btc,4.180181e+07,1.598513e+06,46.565582
eth,6.665769e+08,2.549009e+07,18.032160
usdt,NaN,NaN,7.603567
bnb,5.086336e+09,1.945029e+08,3.020624
xrp,2.095904e+12,8.014796e+10,2.520860
usdc,NaN,NaN,2.383672
steth,NaN,NaN,1.248294
ada,NaN,NaN,0.833110
doge,NaN,NaN,0.812008
sol,NaN,NaN,0.768930


In [61]:
df_global[['total_market_cap','total_volume']].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [56]:
df_global

,total_market_cap,total_volume,market_cap_percentage
btc,4.180181e+07,1.598513e+06,46.565582
eth,6.665769e+08,2.549009e+07,18.032160
usdt,NaN,NaN,7.603567
bnb,5.086336e+09,1.945029e+08,3.020624
xrp,2.095904e+12,8.014796e+10,2.520860
...,...,...,...
xdr,8.181760e+11,3.128728e+10,NaN
xag,4.642986e+10,1.775491e+09,NaN
xau,5.730264e+08,2.191269e+07,NaN
bits,4.180181e+13,1.598513e+12,NaN


In [87]:
df_global.to_csv('./Data/global.csv')

In [129]:
with open('./Data/response_1692752649857.json', 'r') as archivo:
    data = json.load(archivo)

In [143]:
data['data']['market_cap_percentage']

{'btc': 46.5640960218742,
 'eth': 18.03542166409418,
 'usdt': 7.59789395528569,
 'bnb': 3.02227117578155,
 'xrp': 2.5184099716229693,
 'usdc': 2.380593040947237,
 'steth': 1.2496005374866797,
 'ada': 0.8335462673340885,
 'doge': 0.8117388962630163,
 'sol': 0.7706037694637595}

In [139]:
total_market_cap_dict = data['data']['market_cap_percentage']
df_total_market_cap = pd.DataFrame.from_dict(total_market_cap_dict, orient='index', columns=['market_cap_percentage'])

In [140]:
df_total_market_cap.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, btc to sol
Data columns (total 1 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   market_cap_percentage  10 non-null     float64
dtypes: float64(1)
memory usage: 160.0+ bytes
